In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Meta-analysis
This notebook is a replication of the key results of the following meta-analysis for the effect of laser treatment (LLLT) on lymphedema in women with breast cancer:

"Effect of low-level laser therapy on pain and swelling in women with breast cancer-related lymphedema: a systematic review and meta-analysis" *Smoot, B et.al.*, 2014 J Cancer Surviv (2015) 9:287–304 DOI 10.1007/s11764-014-0411-1

Below is the table of change in limb circumference (delta C) over the duration of laser therapy for the laser group and control group, as found in the above paper.

In [3]:
df = pd.DataFrame(np.array([[23,11,10,25],
                          [24,10,10,25],
                          [11.1,373.6,31.33,-29],
                          [13.9,432.1,33.33,-21.8],
                          [9.2,128.4,1.58,3.75],
                          [5.9,164.4,1.73,6.9]]).T,
            columns = pd.Index(['nlaser','ncontrol','meanlaser','meancontrol','sdlaser','sdcontrol'],name='delta C - between groups'),
            index = ['Kozanoglu','Lau','Maiya','Omar']).convert_dtypes()
df['unit'] = ['cm','mL','cm','cm']
display(df)

,nlaser,ncontrol,meanlaser,meancontrol,sdlaser,sdcontrol,unit
Kozanoglu,23,24,11.10,13.90,9.20,5.90,cm
Lau,11,10,373.60,432.10,128.40,164.40,mL
Maiya,10,10,31.33,33.33,1.58,1.73,cm
Omar,25,25,-29.00,-21.80,3.75,6.90,cm


To compute the effect size for each paper, we can use Cohen's D:

$ d = \frac{\mu_1 - \mu_2 }{\sigma_{pooled}} $

where $\mu$ is the group mean and $\sigma_{pooled}$ is the pooled standard deviation. 

The equation for the pooled standard deviation is as follows:

$\sigma_{pooled} = \sqrt{\frac{(n_1-1)\sigma_1^2 + (n_2-1)\sigma_2^2}{n_1 + n_2 - 2}} $

where $n$ is the number of samples per group.

In [31]:
df['pooled_sd'] = np.sqrt(
    ( (df.nlaser-1)*(df.sdlaser)**2 + (df.ncontrol-1)*(df.sdcontrol)**2 )/(df.nlaser+df.ncontrol-2)
)
df['effect_size'] = (df.meanlaser - df.meancontrol)/df.pooled_sd
df[['pooled_sd','effect_size']]

,pooled_sd,effect_size
Kozanoglu,7.692291,-0.364001
Lau,146.559103,-0.399156
Maiya,1.656699,-1.207220
Omar,5.553040,-1.296587


It appears that one of these effect sizes differs from that calculated in the paper by Smoot. 

To get a confidence interval for each effect size, we need the variance for Cohen's D:

$ \sigma_d^2 = \frac{n_1 + n_2}{n_1 \cdot n_2} + \frac{d^2}{2(n_1 + n_2)} $

Note that this is an approximation given by Hedges and Olkin (1985, p86)

The 95% confidence interval is then given by:

$ ( d - z_{95}\cdot\sigma_d, d + z_{95}\cdot\sigma_d ) =  ( d - 1.96\cdot\sigma_d,d + 1.96\cdot\sigma_d ) $

In [34]:
df['variance_ES'] = (df.nlaser + df.ncontrol)/(df.nlaser*df.ncontrol) \
    + df.effect_size**2 / (2*(df.nlaser + df.ncontrol))
df['95% CI'] = df.apply(lambda x:"(%.2f, %.2f)"%(
    x.effect_size - 1.96*np.sqrt(x.variance_ES), x.effect_size + 1.96*np.sqrt(x.variance_ES)),axis=1)
df[['variance_ES','effect_size','95% CI']]

,variance_ES,effect_size,95% CI
Kozanoglu,0.086554,-0.364001,"(-0.94, 0.21)"
Lau,0.194703,-0.399156,"(-1.26, 0.47)"
Maiya,0.236435,-1.207220,"(-2.16, -0.25)"
Omar,0.096811,-1.296587,"(-1.91, -0.69)"


(Taken from https://www.meta-analysis.com/downloads/M-a_f_e_v_r_e_sv.pdf)

The pooled effect size can be obtained under the Fixed Effect model. In this model, the observed effect from study $i$, $T_i$ is determined by the common effect $\mu$ plus the within-study error $\epsilon_i$.

$T_i = \mu + \epsilon_i $

The only source of sampling error under this model is the within studies error. We want to assign more importance to studies that have more information. Here, this is done by assigning weights $w_i$ by the inverse of the variance. 

$ w_i = \frac{1}{v_i} $

The weighted mean $(\bar{T}.)$ is then computed as

$ \bar{T}. = \frac{\sum{w_iT_i}}{\sum{w_i}} $

THe standard error of the combined effect is given by

$ SE(\bar{T}.) = \sqrt{\frac{1}{\sum{w_i}}} $

And finally, the 95% CI is given by:

( $\bar{T}. - 1.96\cdot SE(\bar{T}.) $ ,$\bar{T}. + 1.96\cdot SE(\bar{T}.) $)

In [35]:
pooled_ES = (df.effect_size / df.variance_ES).sum() / (1/df.variance_ES).sum()
pooled_ES_SE = np.sqrt(1/(1/df.variance_ES).sum())
pooled_ES_CI = (pooled_ES - 1.96*pooled_ES_SE, pooled_ES + 1.96*pooled_ES_SE)
print("Pooled Effect Size and 95%% CI: %.3f, (%.2f,%.2f)"%(pooled_ES, *pooled_ES_CI))

Pooled Effect Size and 95% CI: -0.792, (-1.14,-0.44)


As extra information, we can add the standard error for each study:

In [37]:
df['STDERR (pooled)'] = df.pooled_sd / np.sqrt(df.nlaser + df.ncontrol)
display(df)

,nlaser,ncontrol,meanlaser,meancontrol,sdlaser,sdcontrol,unit,pooled_sd,effect_size,variance_ES,95% CI,STDERR (pooled)
Kozanoglu,23,24,11.10,13.90,9.20,5.90,cm,7.692291,-0.364001,0.086554,"(-0.94, 0.21)",1.122036
Lau,11,10,373.60,432.10,128.40,164.40,mL,146.559103,-0.399156,0.194703,"(-1.26, 0.47)",31.981818
Maiya,10,10,31.33,33.33,1.58,1.73,cm,1.656699,-1.207220,0.236435,"(-2.16, -0.25)",0.370449
Omar,25,25,-29.00,-21.80,3.75,6.90,cm,5.553040,-1.296587,0.096811,"(-1.91, -0.69)",0.785318
